In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,39110
2,Huelva,Confirmados PDIA 14 días,3167
3,Huelva,Tasa PDIA 14 días,"617,1444160804411"
4,Huelva,Confirmados PDIA 7 días,1598
5,Huelva,Tasa PDIA 7 dias,"311,3977824112867"
6,Huelva,Total Confirmados,39317
7,Huelva,Curados,34185
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  39110.0


/tmp/ipykernel_3702/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10948.0


/tmp/ipykernel_3702/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3702/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3702/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3702/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 301 personas en los últimos 7 días 

Un positivo PCR cada 149 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,39110.0,1598.0,3167.0,513170.0,311.397782,617.144416,778.0
Huelva-Costa,23013.0,969.0,1997.0,289548.0,334.659538,689.695664,490.0
Huelva (capital),10948.0,478.0,964.0,143837.0,332.320613,670.203077,257.0
Condado-Campiña,12237.0,438.0,777.0,156231.0,280.354091,497.340477,210.0
Sierra de Huelva-Andévalo Central,3484.0,189.0,390.0,67391.0,280.452879,578.712291,76.0
Isla Cristina,2674.0,117.0,222.0,21393.0,546.907867,1037.722620,62.0
Moguer,1680.0,82.0,146.0,21867.0,374.994284,667.672749,46.0
Lepe,2560.0,100.0,205.0,27880.0,358.680057,735.294118,44.0
Punta Umbría,1086.0,80.0,157.0,15355.0,521.002931,1022.468251,37.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,157.0,20.0,39.0,1310.0,1526.717557,2977.099237,7.0
Cumbres Mayores,99.0,31.0,42.0,1749.0,1772.441395,2401.372213,22.0
Cumbres de Enmedio,1.0,1.0,1.0,51.0,1960.784314,1960.784314,NaN
Corteconcepción,34.0,9.0,10.0,536.0,1679.104478,1865.671642,0.0
Santa Ana la Real,19.0,3.0,8.0,475.0,631.578947,1684.210526,0.0
Beas,310.0,25.0,56.0,4341.0,575.904170,1290.025340,11.0
Isla Cristina,2674.0,117.0,222.0,21393.0,546.907867,1037.722620,62.0
Punta Umbría,1086.0,80.0,157.0,15355.0,521.002931,1022.468251,37.0
Cañaveral de León,9.0,2.0,4.0,396.0,505.050505,1010.101010,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Zalamea la Real,106.0,2.0,9.0,3054.0,65.487885,294.695481,1.0,0.222222
Aroche,103.0,2.0,8.0,3054.0,65.487885,261.951539,2.0,0.250000
Nerva,231.0,10.0,31.0,5169.0,193.461018,599.729155,3.0,0.322581
Rosal de la Frontera,43.0,2.0,6.0,1697.0,117.855038,353.565115,0.0,0.333333
Palos de la Frontera,888.0,28.0,78.0,11742.0,238.460228,664.282064,11.0,0.358974
Valverde del Camino,761.0,22.0,60.0,12750.0,172.549020,470.588235,8.0,0.366667
Santa Ana la Real,19.0,3.0,8.0,475.0,631.578947,1684.210526,0.0,0.375000
Niebla,318.0,3.0,8.0,4116.0,72.886297,194.363460,1.0,0.375000
Cartaya,1582.0,35.0,91.0,20083.0,174.276751,453.119554,20.0,0.384615
